In [1]:
#Keras Model Visualization

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam
import os
import numpy as np
import matplotlib.pyplot as plt
batchsize = 10
NUM_EPOCHS = 10
#img_width, img_height = 150, 150
#for VGG 16 use 224 image size
img_width, img_height = 224, 224

Using TensorFlow backend.


## Data
look in TF_sagemaker_install.ipynb  
You'll see the commands to move (kaggle) data from s3 to local directory  
    

In [2]:
train_dir = '/home/ec2-user/SageMaker/data/furniture-images/img/train'
val_dir =   '/home/ec2-user/SageMaker/data/furniture-images/img/val'

train_bed_dir = os.path.join(train_dir, 'bed') 
train_chair_dir = os.path.join(train_dir, 'chair')  
train_sofa_dir = os.path.join(train_dir, 'sofa') 
val_bed_dir = os.path.join(val_dir, 'bed')  
val_chair_dir = os.path.join(val_dir, 'chair')  
val_sofa_dir = os.path.join(val_dir, 'sofa')  


In [3]:
num_bed_train = len(os.listdir(train_bed_dir))
num_chair_train = len(os.listdir(train_chair_dir))
num_sofa_train = len(os.listdir(train_sofa_dir))

num_bed_val = len(os.listdir(val_bed_dir))
num_chair_val = len(os.listdir(val_chair_dir))
num_sofa_val = len(os.listdir(val_sofa_dir))

num_train_images = num_bed_train + num_chair_train + num_sofa_train
num_val_images = num_bed_val + num_chair_val + num_sofa_val
print(num_train_images, num_val_images)

2700 300


In [4]:
train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batchsize)

Found 4024 images belonging to 5 classes.


In [5]:
val_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

val_generator = val_datagen.flow_from_directory(val_dir, 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batchsize)

Found 423 images belonging to 5 classes.


In [6]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


## Create Model

### Memory - OOM Error
I tried running this on a laptop with a GTX 1060 w/ 6 MB, Out of Memory Error!!  
Switched to SageMaker where a p3 has 16 MB (no problem)  



## Model Choice
### custom model - named model
### VGG16 model - named base_model

Choose 1 - PAY CLOSE ATTENTION TO WHAT CELLS TO EXECUTE vs SKIP

custom model: Total params: 714,961,283
VGG16 model:  Total params: 16,292,675

In [ ]:
#BUILD A CUSTOM MODEL - SKIP THIS STEP IF YOU USE VGG16.  FOR CUSTOM MODEL USE THIS STEP AND SKIP NEXT TWO STEPS

model = Sequential([
    Conv2D(96, 11, padding='valid', activation='relu', 
           input_shape=(img_height, img_width ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(256, 5, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(384, 3, padding='same', activation='relu'),
    Conv2D(384, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(1024, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(3)
])


In [7]:
#BUILD VGG - SKIP CUSTOM MODEL STEP IF YOU USE VGG

base_model = VGG16(weights='imagenet', 
                      include_top=False, 
                      input_shape=(img_height, img_width, 3))

In [8]:
#BUILD VGG - SKIP CUSTOM MODEL STEP IF YOU USE VGG
def build_final_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = True

    x = base_model.output
    
    
    x = GlobalAveragePooling2D()(x)
   
    x = Flatten()(x)
    
    
    # Fine-tune from this layer onwards
    fine_tune_at = 100

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable =  False

    
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    final_model = Model(inputs=base_model.input, outputs=predictions)

    return final_model




class_list = ["bed", "chair", "sofa"]
FC_LAYERS = [1024, 1024]
dropout = 0.3

model = build_final_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))

## NOTE - only run one (1) of the following model.compiles
based on your model selection

In [ ]:
#APPLICABLE FOR CUSTOM MODEL
model.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])



In [9]:
#APPLICABLE FOR VGG & INCEPTION
adam = Adam(lr=0.00001)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Page 109l

## Errors:

### VGG16

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[10,3] labels_size=[10,5]
	 [[node loss/dense_2_loss/softmax_cross_entropy_with_logits (defined at /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_1831]

Function call stack:  
distributed_function

### Custom Model:

InvalidArgumentError:  Incompatible shapes: [10,3] vs. [10,5]
	 [[node BroadcastGradientArgs_4 (defined at /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_1698]

Function call stack:  
distributed_function

history = final_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=0,
steps_per_epoch=num_train_images // batchsize,
shuffle=True, validation_data=val_generator,
validation_steps=num_val_images // batchsize)

with

history = final_model.fit(train_generator,epochs=NUM_EPOCHS,steps_per_epoch=num_train_images // batchsize,validation_data=val_generator, validation_steps=num_val_images // batchsize)

In [16]:
print (type(train_generator))
print (NUM_EPOCHS)
print (num_train_images)
print(batchsize)
print (type(val_generator))
print (num_val_images)
print (batchsize)

<class 'keras_preprocessing.image.directory_iterator.DirectoryIterator'>
10
2700
10
<class 'keras_preprocessing.image.directory_iterator.DirectoryIterator'>
300
10


In [23]:
print (train_generator.__len__())
print (train_generator.__getitem__(3))

403
(array([[[[ 63.68357  ,  77.843575 ,  84.94257  ],
         [ 60.5767   ,  74.7367   ,  81.8357   ],
         [ 61.6838   ,  75.8438   ,  82.9428   ],
         ...,
         [ -0.9137573,  14.246239 ,  42.345238 ],
         [  0.2521057,  15.412117 ,  43.511116 ],
         [  1.4179764,  16.57798  ,  44.67698  ]],

        [[ 66.0153   ,  80.1753   ,  87.2743   ],
         [ 60.965324 ,  75.12533  ,  82.22433  ],
         [ 60.129295 ,  74.2893   ,  81.3883   ],
         ...,
         [  1.8504333,  17.01043  ,  45.10943  ],
         [  2.060997 ,  16.58413  ,  45.001564 ],
         [  2.060997 ,  15.806892 ,  44.612938 ]],

        [[ 65.774956 ,  79.93496  ,  87.03396  ],
         [ 62.818657 ,  76.97866  ,  84.07766  ],
         [ 60.43255  ,  74.59255  ,  81.69155  ],
         ...,
         [  2.060997 ,  15.518578 ,  44.46879  ],
         [  0.8618469,  14.021843 ,  43.120842 ],
         [ -1.0812683,  12.078728 ,  41.177727 ]],

        ...,

        [[ 23.060997 ,  26.221   

In [24]:
print (val_generator.__len__())
print (val_generator.__getitem__(3))

43
(array([[[[151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         ...,
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ]],

        [[151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         ...,
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ]],

        [[151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         ...,
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ],
         [151.061    , 138.22101  , 131.32     ]],

        ...,

        [[151.061    , 138.22101  

In [11]:
history = model.fit(train_generator,epochs=NUM_EPOCHS,steps_per_epoch=num_train_images // batchsize,validation_data=val_generator, validation_steps=num_val_images // batchsize)

Train for 270 steps, validate for 30 steps
Epoch 1/10
  1/270 [..............................] - ETA: 3:59:31

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[10,3] labels_size=[10,5]
	 [[node loss/dense_2_loss/softmax_cross_entropy_with_logits (defined at /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_1831]

Function call stack:
distributed_function


In [12]:
print(len(model.layers))

26


In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

img_path = 'furniture_images/test/chair/00000800testchair.jpg'
#img_path = 'furniture_images/test/bed/00000876trainbed.jpg'
#img_path = 'furniture_images/test/sofa/00000803testsofa.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = preprocess_input(img_tensor)

featuremap = model.predict(img_tensor)
plt.imshow(featuremap)
plt.imshow(img_tensor[0])
print (img_tensor.shape)

In [ ]:
from tensorflow.keras.models import Model
layer_outputs = [layer.output for layer in model.layers[:len(model.layers)]]
activation_modelfig = Model(inputs=model.input, outputs=layer_outputs)
activationsfig = activation_modelfig.predict(img_tensor)

In [ ]:
first_layer_activation = activationsfig[0]
print(first_layer_activation.shape)
plt.matshow(first_layer_activation[0, :, :, 2], cmap='viridis')

In [ ]:
for i in range(0,len(model.layers)-8):
    current_layer_activation = activationsfig[i]
    ns = current_layer_activation.shape[-1]
    fig = plt.figure()
    ax1 = fig.add_subplot(131)
    plt.subplot(131)
    plt.imshow(current_layer_activation[0, :, :, 0], cmap='viridis')
    
    ax3 = fig.add_subplot(132)
    plt.subplot(132)
    plt.imshow(current_layer_activation[0, :, :, int(ns/2)], cmap='viridis')
    
    ax5 = fig.add_subplot(133)
    plt.subplot(133)
    plt.imshow(current_layer_activation[0, :, :, ns-1], cmap='viridis')
    